# And now... ORM!

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("postgresql://alchemist@alchemist_db/alchemist", echo=True)

We start the configurational process by describing the database tables we’ll be dealing with, and then by defining our own classes which will be mapped to those tables. 

We performe those two tasks together, using a system known as `Declarative`, which allows us to create classes that include directives to describe the actual database table they will be mapped to.

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

# `__tablename__` and at least one `primary_key` must be provided
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    
    def __repr__(self):
        return '<User(name={}, fullname={})>'.format(self.name, self.fullname)

In [ ]:
User.__table__

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
seven = User(name='7of9', fullname='Seven of Nine')

In [ ]:
print(seven.name, ', id: ', seven.id)

### Talking to the database using a Session

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
print(Session)

In [ ]:
session = Session()
print(session)

In [ ]:
session.add(seven)  # pending (no SQL issued)

In [ ]:
# we need a `flush`, and it will be issued automagically if needed
from_db = session.query(User).filter_by(name='7of9').first()

In [ ]:
print(from_db)

In [ ]:
seven is from_db  # identity mapping!

In [ ]:
session.add_all([
    User(name='ellen', fullname='Ellen Ripley'),    
    User(name='cameron', fullname='Cameron Phillips'),
    User(name='sarah', fullname='Sarah Connor'),
])

# update already existing object
seven.name = 'seven'

In [ ]:
print(session.new)
session.dirty

In [ ]:
session.commit()

In [ ]:
session.dirty

In [ ]:
# objects moved through `transient`, `pending` and `persistent` states
# read the details: http://docs.sqlalchemy.org/en/rel_1_0/orm/session_state_management.html#session-object-states

In [ ]:
omnipotent = User(name='q', fullname='?')
session.add(omnipotent)
seven.name = 'seeeven'

In [ ]:
session.query(User).filter(User.name.in_(['seeeven', 'q'])).all()

In [ ]:
session.rollback()
print(seven.name)
omnipotent in session

## Querying

In [ ]:
for obj in session.query(User).order_by(User.name):
    print(obj.name, obj.fullname)

In [ ]:
# being explicit with arguments to `query` gets us tuples (`KeyedTuple`)
for name, full in session.query(User.name, User.fullname):
    print(name, full)

In [ ]:
# LIMIT and OFFSET
for usr in session.query(User).order_by(User.id)[1:3]:
    print(usr)

In [ ]:
# `filter_by` (keyword args)
for name, in session.query(User.name).filter_by(fullname='Ellen Ripley'):
    print(name)

In [ ]:
# `filter` (SQL expression)
for name, in session.query(User.name).filter(User.fullname=='Ellen Ripley'):
    print(name)

### lists and scalars

In [ ]:
query = session.query(User).filter(User.name.like('%en%')).order_by(User.id)

In [ ]:
query.all()

In [ ]:
query.first()

In [ ]:
query.one()

In [ ]:
user = query.filter(User.id==666).one()

In [ ]:
query = session.query(User.id).filter(User.name=='seven')
query.scalar()  # returns the first column of the single row

### count

In [ ]:
session.query(User).filter(User.name.like('%en%')).count()